In [82]:
from leagues.models import Competition,Season
from base_app.helpers import best_fuzzy_match
from games.models import Game
from datetime import datetime
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
import os

In [84]:
EV_PATH = "D:\MetaData"
ST_PATH = "D:\MetaData-shot"

event_files = []
for base,_,files in os.walk(EV_PATH):
    for f in files:
        if 'shot' not in f :
            event_files.append(f"{base}\{f}")

shot_files = []
for base,_,files in os.walk(ST_PATH):
    for f in files:
        # print(f)
        shot_files.append(f"{base}\{f}")

event_files = list(set(event_files))
shot_files = list(set(shot_files))
print(f"{len(event_files)} : {event_files[:3]}")
print(f"{len(shot_files)} : {shot_files[:3]}")

547 : ['D:\\MetaData\\CONMEBOL\\Venezuela\\Primera_División_2018_events.xlsx', 'D:\\MetaData\\CONCACAF\\CONCACAF\\Concacaf_Champions_Cup_2019_events.xlsx', 'D:\\MetaData\\CONMEBOL\\Colombia\\Primera_A_2022_events.xlsx']
199 : ['D:\\MetaData-shot\\CONMEBOL\\Argentina\\Liga_Profesional_2024_shots.xlsx', 'D:\\MetaData-shot\\UEFA\\Austria\\Bundesliga_2021_2022_shots.xlsx', 'D:\\MetaData-shot\\UEFA\\Scotland\\Premiership_2024_2025_shots.xlsx']


In [74]:
len(event_files),len(shot_files)

(547, 0)

In [66]:
map_example = []
non_mapped_season = []
for season in Season.objects.all():
    conf = season.competition.confederation
    region = season.competition.country
    
    e_name = season.competition.name_scoresaway.replace(' ','_').replace(".","")
    s_name = season.competition.name_fotmob.replace(' ','_').replace(".","")
    
    e_s_name = season.name.replace('/','_').replace(' ','_')
    s_s_name = season.name_fotmob.replace('/','_').replace(' - ','_')

    if season.competition_id == 27 :
        e_search_str = f"D:\\MetaData\\{conf}\\{region}\\{e_name}_{e_s_name}_events.xlsx"
        s_search_str = [f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{e_s_name}_Apertura_shots.xlsx",
                        f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{e_s_name}_Clausura_shots.xlsx"]
        print(e_s_name,"|",s_s_name,e_search_str in event_files,s_search_str[0] in shot_files,s_search_str[1] in shot_files)
        if e_search_str in event_files and s_search_str[0] in shot_files and s_search_str[1] in shot_files  :
            map_example.append((season,e_search_str,"__||__".join(s_search_str)))
    else:
        continue
        e_search_str = f"D:\\MetaData\\{conf}\\{region}\\{e_name}_{e_s_name}_events.xlsx"
        s_search_str = f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{s_s_name}_shots.xlsx"

    if e_search_str in event_files and s_search_str in shot_files  :
        map_example.append((season,e_search_str,s_search_str))
    else:
        non_mapped_season.append((season,e_search_str))

2024_2025 | 2024_2025_Apertura True True True
2023_2024 | 2023_2024_Apertura True True True
2022_2023 | 2022_2023_Apertura True True True
2021_2022 | 2021_2022_Apertura True True True
2020_2021 | 2020_2021_Apertura True True True
2019_2020 | 2019_2020_Apertura True False False
2018_2019 | 2018_2019_Apertura True False False
2017_2018 | 2017_2018_Apertura True False False
2024_2025 | 2024_2025_Clausura True True True
2023_2024 | 2023_2024_Clausura True True True
2022_2023 | 2022_2023_Clausura True True True
2021_2022 | 2021_2022_Clausura True True True
2020_2021 | 2020_2021_Clausura True True True
2019_2020 | 2019_2020_Clausura True False False
2018_2019 | 2018_2019_Clausura True False False


In [57]:
from datetime import timedelta
                               # e_df,s_df,event_teams,shot_teams,mapping_dict,8,return_counts=True
def match_and_count_for_season(e_df,s_df,event_teams,shot_teams,hrs,return_counts=False):
    max_diff_hours = hrs
    results = []
    merged = []
    corr = 0
    wrong = 0
    ups = 0
    not_req = 0
    min_dt_of_event = s_df['datetime'].min()
    for i, row in e_df.iterrows():
        e_home = row['home_team']
        e_away = row['away_team']
        e_date = row['datetime']

        if (e_date-min_dt_of_event).total_seconds()/3600 < -8 :
            not_req+=1
            continue
        fuzzy_home = best_fuzzy_match(e_home, shot_teams,threshold_multi=75)
        fuzzy_away = best_fuzzy_match(e_away, shot_teams,threshold_multi=75)
        
        # Get top fuzzy keys (most probable names)
        possible_homes = list(fuzzy_home.keys())
        possible_aways = list(fuzzy_away.keys())
        
        # Check all possible home-away combinations in df
        mask = (
            (s_df['home_team'].isin(possible_homes)) &
            (s_df['away_team'].isin(possible_aways)) &
            (abs(s_df['datetime'] - e_date) <= timedelta(hours=max_diff_hours))
        )
        
        matches = s_df[mask]
        if matches.shape[0] >1:
            print("fakk, there are more ... do something kent saab")
            # print(f"\n{e_home} -> {fuzzy_home}")
            # print(f"{e_away} -> {fuzzy_away}")
            # print(f"datetime : {e_date} \n\n")
            
            # ups+=1
            fuzzy_home = best_fuzzy_match(e_home, shot_teams,threshold_multi=85)
            fuzzy_away = best_fuzzy_match(e_away, shot_teams,threshold_multi=85)
            
            # Get top fuzzy keys (most probable names)
            possible_homes = list(fuzzy_home.keys())
            possible_aways = list(fuzzy_away.keys())
            
            # Check all possible home-away combinations in df
            mask = (
                (s_df['home_team'].isin(possible_homes)) &
                (s_df['away_team'].isin(possible_aways)) &
                (abs(s_df['datetime'] - e_date) <= timedelta(hours=max_diff_hours))
            )
            
            matches = s_df[mask]
            if matches.shape[0] > 1 :
                print("Fadd lo bhai hun dooji waar!!!!!!!!!!")
                print(f"\n{e_home} -> {fuzzy_home}")
                print(f"{e_away} -> {fuzzy_away}")
                print(f"datetime : {e_date} \n\n")
                ups+=1
                
        # print(f"Found {matches.shape[0]} matched for the fuzzy map combinations")
        # print(f"\n{e_home} -> {fuzzy_home}")
        # print(f"{e_away} -> {fuzzy_away}")
        # print(f"datetime : {e_date}")
        if not matches.empty:
            correct_match = matches.iloc[0]
                    
            temp = {}
            temp['e_home_team'] = e_home
            temp['s_home_team'] = correct_match['home_team']
            
            temp['e_away_team'] = e_away
            temp['s_away_team'] = correct_match['away_team']
        
            temp['e_datetime'] = e_date
            temp['s_datetime'] = correct_match['datetime']
        
            temp['event_url'] = row['event_url']
            temp['shot_url'] = correct_match['shot_url']
            merged.append(temp) 
            corr+=1
        else:
            # print("\n Adding Blanks \n")
            temp = {}
            temp['e_home_team'] = e_home
            temp['s_home_team'] = "NA"
            
            temp['e_away_team'] = e_away
            temp['s_away_team'] = "NA"
        
            temp['e_datetime'] = e_date
            temp['s_datetime'] = "NA"
        
            temp['event_url'] = row['event_url']
            temp['shot_url'] = "NA"
            merged.append(temp) 
            if fuzzy_home == {} or fuzzy_away == {} :
            
                # print(row)
                # print()
                print(f"\nF : {e_home} -> {fuzzy_home}")
                print(f"{e_away} -> {fuzzy_away}")
                print(f"datetime : {e_date}")
                print(f"Diff : {e_date-min_dt_of_event} \n")
            else:
                print(f" S : \n{e_home} -> {fuzzy_home}")
                print(f"{e_away} -> {fuzzy_away}")
                print(f"datetime : {e_date}")
                print(f"Min Dt : {min_dt_of_event}")
            
            wrong+=1
            print("❌ No close match found.")
        # if corr+wrong+ups != len(merged):
        #     print(f"{corr}+{wrong}+{ups} != {len(merged)}\n")
        #     print(row)
    if return_counts :
        return corr,wrong,ups
    return pd.DataFrame(merged),corr,wrong,ups
# results_df = pd.DataFrame(results)

In [58]:
merged_data_collection = []
BASE_META_DATA="D:/Merged_Items"
err,done=0,0

In [64]:
Map_example = [x for x in map_example if 'Liga MX' == x[0].competition.competition_name]
               # and x[0].name in ["2020/2021", "2024/2025", "2022/2023", "2019/2020", '2018/2019']]

len(Map_example),Season.objects.filter(competition__id=27).count()

(10, 15)

In [60]:
for i,(season,event_path,shot_path) in enumerate(map_example):
    print(f"Working on {i}/{len(map_example)} | Completed : {done} | Failed {err} | {season.name}")
    try :
        e_df = pd.read_excel(event_path).sort_values(['datetime','home_team','away_team'])
        # s_df = pd.read_excel(shot_path).sort_values(['datetime','home_team','away_team'])
        s_df = pd.concat([pd.read_excel(x) for x in shot_path.split("__||__")])
        event_teams = list(set(e_df['home_team'].to_list()+e_df['away_team'].to_list()))
        shot_teams = list(set(s_df['home_team'].to_list()+s_df['away_team'].to_list()))
    
        conf = season.competition.confederation
        country = season.competition.country
        name = season.competition.competition_name
        df,corr,wrong,ups = match_and_count_for_season(e_df,s_df,event_teams,shot_teams,11,return_counts=False)
        # print("Merging Done !!")
        print(f"df : {df.shape[0]}/{e_df.shape[0]}/{s_df.shape[0]} | corr : {corr} | wrong : {wrong} | ups : {ups} \n")
        # continue
        os.makedirs(BASE_META_DATA, exist_ok=True)
        os.makedirs(f"{BASE_META_DATA}/{conf}", exist_ok=True)
        os.makedirs(f"{BASE_META_DATA}/{conf}/{country}", exist_ok=True)
        merge_path = f"{BASE_META_DATA}/{conf}/{country}/{name.replace(' ','_')}__{season.name_fotmob.replace('/','_').replace(' - ','_').replace(' ','_').replace('-','_')}__Merged.xlsx"
        df.to_excel(merge_path,index=False)
        
        temp = {}
        temp['competition'] = f"{season.competition.competition_name}"
        temp['season'] = f"{season.name_fotmob}"
        
        
        temp['events_count'] = e_df.shape[0]
        temp['shot_count'] = s_df.shape[0]
        temp['merged_count'] = s_df.shape[0]
    
        temp['events_path'] = event_path
        temp['shot_path'] = shot_path
        temp['merged_path'] = merge_path
        merged_data_collection.append(temp)
        print(f"Completed '{season.competition.competition_name} | {season.name_fotmob}'")
        done+=1
    except Exception as e :
        print(f"Failed '{season.competition.competition_name} | {season.name_fotmob}'")
        print(e)
        err+=1

Working on 0/199 | Completed : 0 | Failed 0 | 2024/2025
df : 176/176/176 | corr : 176 | wrong : 0 | ups : 0 

Completed 'A-League | 2024/2025'
Working on 1/199 | Completed : 1 | Failed 0 | 2023/2024
df : 169/169/169 | corr : 169 | wrong : 0 | ups : 0 

Completed 'A-League | 2023/2024'
Working on 2/199 | Completed : 2 | Failed 0 | 2022/2023
df : 163/163/163 | corr : 163 | wrong : 0 | ups : 0 

Completed 'A-League | 2022/2023'
Working on 3/199 | Completed : 3 | Failed 0 | 2021/2022
df : 163/163/163 | corr : 163 | wrong : 0 | ups : 0 

Completed 'A-League | 2021/2022'
Working on 4/199 | Completed : 4 | Failed 0 | 2020/2021
df : 161/161/161 | corr : 161 | wrong : 0 | ups : 0 

Completed 'A-League | 2020/2021'
Working on 5/199 | Completed : 5 | Failed 0 | 2019/2020
df : 147/147/148 | corr : 147 | wrong : 0 | ups : 0 

Completed 'A-League | 2019/2020'
Working on 6/199 | Completed : 6 | Failed 0 | 2018/2019
df : 140/140/140 | corr : 140 | wrong : 0 | ups : 0 

Completed 'A-League | 2018/2019'

In [61]:
pd.DataFrame(merged_data_collection).to_excel("V1_Merged_Info.xlsx",index=False)

In [47]:
home = "CA Belgrano"
away = "Club Estudiantes de La Plata"
e_date = datetime.strptime("2023-08-18 04:00:00","%Y-%m-%d %H:%M:%S")

In [50]:
fuzzy_home = best_fuzzy_match(home, shot_teams)
fuzzy_away = best_fuzzy_match(away, shot_teams)

# Get top fuzzy keys (most probable names)
possible_homes = list(fuzzy_home.keys())
possible_aways = list(fuzzy_away.keys())
print(possible_homes,"\n",possible_aways)
# Check all possible home-away combinations in df
mask = (
    (s_df['home_team'].isin(possible_homes+possible_aways))
    & (s_df['away_team'].isin(possible_homes+possible_aways))
    # & (abs(s_df['datetime'] - e_date) <= timedelta(hours=11))
)

matches = s_df[mask]
# print(abs(list(matches['datetime'])[0] - e_date),e_date)
matches

['Belgrano'] 
 ['Estudiantes']


,home_team,home_score,away_team,away_score,datetime,shot_url
351,Estudiantes,4,Belgrano,0,2023-07-22 21:00:00,https://www.fotmob.com/matches/belgrano-vs-estudiantes/3db8a5#4087299
